In [1]:
from remote_sensing_ddpm.datasets.uc_merced_land_use.uc_merced_dataset import UCMerced

uc_merced_data_root = "/Users/alexanderlontke/Documents/Uni/St. Gallen/SS_23/master_thesis/data/UCMerced_LandUse/Images"

dataset = UCMerced(
    data_root=uc_merced_data_root,
    phase="train",
    split_ratios={"train": 1.0}
)

Loading Classes:: 100%|██████████| 21/21 [00:02<00:00,  8.33it/s]


In [2]:
count = 0
for i, entry in enumerate(dataset):
    if not entry["image"].shape == (3, 256, 256):
        print(entry["image"].shape)
        count +=1
print(count)

0


In [9]:
print(entry)

{'image': tensor([[[-0.7569, -0.6000, -0.5843,  ..., -0.1373, -0.2000, -0.5765],
         [-0.3882, -0.2471, -0.3647,  ..., -0.3647, -0.4196, -0.5922],
         [-0.3333, -0.2784, -0.4588,  ..., -0.7725, -0.7647, -0.7098],
         ...,
         [-0.0588, -0.2314, -0.4118,  ..., -0.0980, -0.6157, -0.4667],
         [ 0.0745, -0.2627, -0.6235,  ..., -0.0745, -0.4824, -0.2784],
         [ 0.1686, -0.1922, -0.6627,  ...,  0.0118, -0.1373, -0.2784]],

        [[-0.6392, -0.4902, -0.4588,  ...,  0.0824, -0.0039, -0.4275],
         [-0.2549, -0.0824, -0.2000,  ..., -0.1373, -0.2078, -0.4196],
         [-0.1843, -0.0824, -0.2706,  ..., -0.6314, -0.6314, -0.5686],
         ...,
         [-0.0431, -0.1843, -0.2941,  ...,  0.0353, -0.5294, -0.3647],
         [ 0.0510, -0.2627, -0.5686,  ...,  0.0510, -0.3647, -0.1451],
         [ 0.1216, -0.2157, -0.6471,  ...,  0.1686,  0.0118, -0.1216]],

        [[-0.6392, -0.5216, -0.4902,  ..., -0.0667, -0.1294, -0.5059],
         [-0.2706, -0.1529, -0.2706

In [1]:
def conv_output(h, kernel, stride):
    return (h-kernel)/stride +1

In [2]:
conv_output(256, 5, 1)

252.0

In [14]:
conv_output(252, 4, 2)

125.0

In [15]:
conv_output(125,4,1)

122.0

In [16]:
conv_output(122,4,2)

60.0

In [17]:
conv_output(60,5,1)

56.0

In [19]:
conv_output(56,4,2)

27.0

In [8]:
from remote_sensing_ddpm.downstream_tasks.models.non_linear_head import NonLinearHead

model = NonLinearHead(input_size=65536, output_size=8)

In [9]:
sum(p.numel() for p in model.parameters())

269490440

In [10]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

269490440

In [11]:
269490440 / 1e6

269.49044

In [12]:
model

NonLinearHead(
  (downstream_layer): Sequential(
    (0): Linear(in_features=65536, out_features=4096, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4096, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=8, bias=True)
    (5): Sigmoid()
  )
)

In [13]:
sum(p.numel() for p in model.downstream_layer[0].parameters())

268439552

In [14]:
sum(p.numel() for p in model.downstream_layer[2].parameters())

1048832

In [15]:
sum(p.numel() for p in model.downstream_layer[4].parameters())

2056

In [30]:
SCHEIBENREIF_ET_AL_SEGMENTATION = {
    ("Forest", "mean"): [62],
    ("Forest", "std"): [2],
    ("Shrubland", "mean"): [48],
    ("Shrubland", "std"): [1],
    ("Grassland", "mean"): [6],
    ("Grassland", "std"): [0],
    ("Wetland", "mean"): [16],
    ("Wetland", "std"): [1],
    ("Cropland", "mean"): [52],
    ("Cropland", "std"): [2],
    ("Urban", "mean"): [82],
    ("Urban", "std"): [0],
    ("Barren", "mean"): [36],
    ("Barren", "std"): [1],
    ("Water", "mean"): [99],
    ("Water", "std"): [0],
}

In [31]:
ml_baselines = [
    "val/jaccardindexadapter_Urban_Built-up",
    "val/jaccardindexadapter_Barren",
    "val/jaccardindexadapter_Wetlands",
    "val/jaccardindexadapter_Shrubland",
    "val/jaccardindexadapter_Croplands",
    "val/jaccardindexadapter_Forest",
    "val/jaccardindexadapter_Grassland",
    "val/jaccardindexadapter_Water",
]

In [32]:
baseline_name = "Scheibenreif et al."
target = {
    (baseline_name, "mean"): {},
    (baseline_name, "std"): {},
}
for (class_name, agg), v in SCHEIBENREIF_ET_AL_SEGMENTATION.items():
    metric_name = [x for x in ml_baselines if class_name in x][0]
    target[(baseline_name, agg)][metric_name] = v[0]

In [33]:
target

{('Scheibenreif et al.', 'mean'): {'val/jaccardindexadapter_Forest': 62,
  'val/jaccardindexadapter_Shrubland': 48,
  'val/jaccardindexadapter_Grassland': 6,
  'val/jaccardindexadapter_Wetlands': 16,
  'val/jaccardindexadapter_Croplands': 52,
  'val/jaccardindexadapter_Urban_Built-up': 82,
  'val/jaccardindexadapter_Barren': 36,
  'val/jaccardindexadapter_Water': 99},
 ('Scheibenreif et al.', 'std'): {'val/jaccardindexadapter_Forest': 2,
  'val/jaccardindexadapter_Shrubland': 1,
  'val/jaccardindexadapter_Grassland': 0,
  'val/jaccardindexadapter_Wetlands': 1,
  'val/jaccardindexadapter_Croplands': 2,
  'val/jaccardindexadapter_Urban_Built-up': 0,
  'val/jaccardindexadapter_Barren': 1,
  'val/jaccardindexadapter_Water': 0}}

In [29]:
pd.DataFrame(target)

Scheibenreif et al.    
                                                    mean std
val/multilabelf1score_Forest                          79   2
val/multilabelf1score_Shrubland                       40   3
val/multilabelf1score_Grassland                       47   6
val/multilabelf1score_Wetlands                        24   2
val/multilabelf1score_Croplands                       70   3
val/multilabelf1score_Urban_Built-up                  80   2
val/multilabelf1score_Barren                          34   3
val/multilabelf1score_Water                           97   0